## Import packages

In [1]:
! pip install nltk numpy pandas unidecode scikit-learn tqdm
! pip install langchain langchain-core langchain-community langchain_experimental langchain-chroma langchain_mistralai langgraph

In [2]:
import os
import nltk
import string
import numpy as np
import pandas as pd
from unidecode import unidecode
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from pathlib import Path
import pickle

from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.llms import Ollama
from langgraph.graph import END, StateGraph

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess(corpus: str) -> str:
  corpus = corpus.lower()
  stopset = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('russian') + list(string.punctuation)
  tokens = nltk.word_tokenize(corpus)
  tokens = [t for t in tokens if t not in stopset]
  tokens = [lemmatizer.lemmatize(t) for t in tokens]
  corpus = ' '.join(tokens)
  corpus = unidecode(corpus)
  return corpus

In [5]:
embeddings = OllamaEmbeddings(model='llama3.1')
store = LocalFileStore("./.embeddings_cache")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, store, namespace=embeddings.model
)

In [6]:
def embeddings_cosine_sim_metric(expected_answers: list[str], predicted_answers: list[str]) -> float:
  results = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    expected_embedding = np.array(cached_embeddings.embed_query(expected_answer))
    predicted_embedding = np.array(cached_embeddings.embed_query(predicted_answer))

    sim = cosine_similarity(
      expected_embedding.reshape(1, -1),
      predicted_embedding.reshape(1, -1),
    )[0][0]

    results.append(sim)

  return np.mean(results)

In [7]:
docs_dir = Path('./docs')
docs_cache_dir = Path('./.docs_cache')
raw_docs_pkl_path = docs_cache_dir / 'parsed_docs_cache.pkl'

docs = None

if os.path.exists(raw_docs_pkl_path):
  with open(raw_docs_pkl_path, 'rb') as f:
    docs = pickle.load(f)
else:
  docs = []
  for file in docs_dir.iterdir():
    docs.extend(PDFMinerLoader(file).load())

  with open(raw_docs_pkl_path, 'wb') as f:
    pickle.dump(docs, f)

In [8]:
from langchain_experimental.text_splitter import SemanticChunker

splitted_docs_pkl_path = docs_cache_dir / 'splitted_docs_cache.pkl'

if os.path.exists(splitted_docs_pkl_path):
  with open(splitted_docs_pkl_path, 'rb') as f:
    splitted_docs = pickle.load(f)
else:
  text_splitter = SemanticChunker(cached_embeddings, breakpoint_threshold_type="gradient")
  splitted_docs = text_splitter.create_documents([doc.page_content for doc in docs])

  with open(splitted_docs_pkl_path, 'wb') as f:
    pickle.dump(docs, f)

JSONDecodeError: Expecting ',' delimiter: line 1 column 61440 (char 61439)

In [27]:
vectorstore = Chroma.from_documents(
  documents=splitted_docs,
  embedding=cached_embeddings,
)
retriever = vectorstore.as_retriever()

In [9]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from typing import Literal
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class RouteQuery(BaseModel):
  data_source: Literal["vectorstore", "websearch"] = Field(
    description="Given a user question choose to route it to web search or a vectorstore.",
  )

# LLM with function call
llm = Ollama(model='llama3.1', temperature=0)
parser = PydanticOutputParser(pydantic_object=RouteQuery)
route_parser = OutputFixingParser(parser=parser, llm=llm)

# Prompt
route_template = """
You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to neurobiology and medicine.
Use the vectorstore for questions on these topics. For all else, use web-search.

{format_instructions}

{question}
"""
route_prompt = PromptTemplate(
  template=route_template,
  input_variables=["question"],
  partial_variables={"format_instructions": route_parser.get_format_instructions()},
)

question_router = route_prompt | llm | route_parser
print(question_router.invoke({"question": "Who will the Bears draft first in the NFL draft?"}))
print(question_router.invoke({"question": "What is the order of the cranial nerves?"}))

data_source='websearch'
data_source='vectorstore'


In [366]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class GradeDocuments(BaseModel):
  binary_score: str = Field(description="Documents are relevant to the question, 'yes' or 'no'")

llm = Ollama(model='llama3.1', temperature=0)
parser = PydanticOutputParser(pydantic_object=GradeDocuments)
docs_grade_parser = OutputFixingParser(parser=parser, llm=llm)

docs_grade_template = """
You are a grader assessing relevance of a retrieved document to a user question. \n
If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.

{format_instructions}

Retrieved document:
{document}

User question:
{question}
"""
docs_grade_prompt = PromptTemplate(
  template=docs_grade_template,
  input_variables=["document", 'question'],
  partial_variables={"format_instructions": docs_grade_parser.get_format_instructions()},
)

docs_grade_grader = docs_grade_prompt | llm | docs_grade_parser

In [367]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class GradeHallucinations(BaseModel):
  binary_score: str = Field(description="Answer is grounded in the facts, 'yes' or 'no'")

# LLM with function call
llm = Ollama(model='llama3.1', temperature=0)
parser = PydanticOutputParser(pydantic_object=GradeHallucinations)
hallucination_parser = OutputFixingParser(parser=parser, llm=llm)

# Prompt
hallucination_template = """
You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."

{format_instructions}

Set of facts:
{documents}

LLM generation:
{generation}
"""
hallucination_prompt = PromptTemplate(
  template=hallucination_template,
  input_variables=["question", 'generation'],
  partial_variables={"format_instructions": hallucination_parser.get_format_instructions()},
)

hallucination_grader = hallucination_prompt | llm | hallucination_parser
print(hallucination_grader.invoke({'documents': ['Sky is blue'], 'generation': 'The color of the sky is blue'}))

binary_score='yes'


In [368]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class GradeAnswer(BaseModel):
  binary_score: str = Field(description="Answer addresses the question, 'yes' or 'no'")

# LLM with function call
llm = Ollama(model='llama3.1', temperature=0)
parser = PydanticOutputParser(pydantic_object=GradeAnswer)
grade_parser = OutputFixingParser(parser=parser, llm=llm)

# Prompt
grade_template = """
You are a grader assessing whether an answer addresses / resolves a question. \n
Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question.

{format_instructions}

User question:
{question}

LLM generation:
{generation}
"""
grade_prompt = PromptTemplate(
  template=grade_template,
  input_variables=["question", 'generation'],
  partial_variables={"format_instructions": grade_parser.get_format_instructions()},
)

answer_grader = grade_prompt | llm | grade_parser
print(answer_grader.invoke({"question": "What is the order of the cranial nerves?", 'generation': 'I do not know.'}))

binary_score='no'


In [369]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

prompt = hub.pull('rlm/rag-prompt')
# LLM
llm = Ollama(model='llama3.1', temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": '', "question": 'What is the order of the cranial nerves ?'})
print(generation)

The order of the 12 cranial nerves is as follows:

I. Olfactory nerve
II. Optic nerve
III. Occulomotor nerve
IV. Trochlear nerve
V. Trigeminal nerve
VI. Abducens nerve
VII. Facial nerve
VIII. Auditory (or vestibulocochlear) nerve
IX. Glossopharyngeal nerve
X. Vagus nerve
XI. Spinal accessory nerve
XII. Hypoglossal nerve


In [370]:
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = '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'
web_search_tool = TavilySearchResults(k=5)

In [371]:
from typing_extensions import TypedDict
from typing import List
from langchain.schema import Document

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """
    documents: List[str]
    web_search: str
    question: str
    generation: str
    generations_num: int

def route_question(state):
    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.data_source == 'websearch':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source.data_source == 'vectorstore':
        print("---ROUTE QUESTION TO VECTOR STORE---")
        return "vectorstore"

def retrieve(state):
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def grade_documents(state):
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for index, d in enumerate(documents):
        print(f'---GRADE DOCUMENT ({index + 1}/{len(documents)})---')
        print(d.page_content)
        score = docs_grade_grader.invoke({"question": question, "document": d.page_content})
        print('score', score)
        grade = score.binary_score
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue

    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def decide_to_generate(state):
    print("---ASSESS GRADED DOCUMENTS---")
    web_search = state["web_search"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

def web_search(state):
    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get('documents')

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    print('Web results', web_results)
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

def generate(state):
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    generations_num = state.get('generations_num', 0)

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {
        "documents": documents,
        "question": question,
        "generation": generation,
        'generations_num': generations_num + 1,
    }

def grade_generation(state):
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    generations_num = state["generations_num"]

    if generations_num >= 2:
        return "useful"

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

workflow = StateGraph(GraphState)
workflow.add_node("websearch", web_search)
workflow.add_node("generate", generate)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": 'websearch',
        "vectorstore": "retrieve",
    },
)
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

app = workflow.compile()

In [372]:
app.invoke({"question": 'What is the order of the cranial nerves?'})

---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
To this end, some basic facts about nerve cells are presented and a few
useful formulations are introduced.
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
To this end, some basic facts about nerve cells are presented and a few
useful formulations are introduced.
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
To this end, some basic facts about nerve cells are presented and a few
useful formulations are introduced.
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
This can be accomplished either by pumping positively charged potassium
ions out of the postsynaptic cell or negatively charged chloride ions into it.
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
Web results Anatomy. Cranial nerves

{'documents': [Document(page_content="Anatomy. Cranial nerves are the 12 nerves of the peripheral nervous system that emerge from the foramina and fissures of the cranium.Their numerical order (1-12) is determined by their skull exit location (rostral to caudal). All cranial nerves originate from nuclei in the brain.Two originate from the forebrain (Olfactory and Optic), one has a nucleus in the spinal cord (Accessory) while the ...\nThe numbering of the cranial nerves is based on the order in which they emerge from the brain and brainstem, from front to back. [2] The terminal nerves (0), olfactory nerves (I) and optic nerves (II) emerge from the cerebrum, and the remaining ten pairs arise from the brainstem, which is the lower part of the brain. [3]\nThe cranial nerves are numbered by their location on the brainstem (superior to inferior, then medial to lateral) and the order of their exit from the cranium (anterior to posterior) (Figures 1 & 2). By Patrick J. Lynch, medical illustrat

In [373]:
qa_df = pd.read_csv('brainscape.csv')[:10]
qa_df

,question,answer
0,What are the afferent cranial nerve nuclei?,Trigeminal sensory nucleus- fibres carry gener...
1,What is the order of the cranial nerves ?,1-olfactory\n2-optic\n3-oculomotor\n4-trochlea...
2,What are the efferent cranial nerve nuclei?,Edinger-westphal nucleus\nOculomotor nucleus\n...
3,Which nuclei share the embryo logical origin -...,Oculomotor nucleus Trochlear nucleus Abducens ...
4,Which nuclei share the embryo logical origin- ...,Trigeminal motor nucleus Facial motor nucleus ...
5,Which nuclei share the embryo logical origin- ...,Edinger-Westphal nucleus Superior and inferior...
6,Which cranial nerves are sensory ?,Olfactory Optic Vestibulocochlear
7,Which cranial nerves are motor?,Oculomotor\nTrochlear \nAbducens\nAccessory\nH...
8,Which of the cranial nerves have both sensory ...,TrigeminalFacial GlossopharyngealVagus
9,What is the function of cranial nerve 1?,The olfactory nerve is a sensory fibre for the...


In [374]:
questions = list(qa_df['question'].tolist())
expected_answers = list(qa_df['answer'].tolist())
predicted_answers = []

for index, question in tqdm(enumerate(questions)):
  answer = app.invoke({'question': question})['generation']
  predicted_answers.append(answer)

# Evaluate metrics
embeddings_cosine_sim_metric(expected_answers, predicted_answers)

0it [00:00, ?it/s]

---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
What other complications of causality are anticipated?
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
Web results The cranial nerve nuclei are a series of bilateral grey matter motor and sensory nuclei located in the midbrain, pons and medulla that are the collections of afferent and efferent cell bodies for many of the cranial nerves.. Some nuclei are small and contribute to a single cranial nerve, such as some of the motor nuclei. Other nuclei, h

1it [00:14, 14.07s/it]

---DECISION: GENERATION ADDRESSES QUESTION---
---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
What other complications of causality are anticipated?
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
Web results Anatomy. Cranial nerves are the 12 nerves of the peripheral nervous system that emerge from the foramina and fissures of the cranium.Their numerical order (1-12) is determined by their skull exit location (rostral to caudal). All cranial nerves originate from nuclei in the brain.Two originat

2it [00:35, 18.51s/it]

...

---DECISION: GENERATION ADDRESSES QUESTION---
---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
biological molecules be described?
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
What other complications of causality are anticipated?
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
Web results Cranial nerves: facial nerve (inferior view) Finally, the oculomotor nerve, the trochlear nerve, the mandibular branch of the trigeminal nerve (V3), the abducens nerve, the facial nerve, the glossopharyngeal nerve, the vagus nerve, the spinal accessory nerve, and the hypoglossal nerve are re

8it [02:11, 16.68s/it]

---DECISION: GENERATION ADDRESSES QUESTION---
---ROUTE QUESTION---
---ROUTE QUESTION TO VECTOR STORE---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE DOCUMENT (1/4)---
biological molecules be described?


8it [02:15, 16.90s/it]


AttributeError: 'NoneType' object has no attribute 'invoke'